In [1]:
import pandas as pd
import numpy as np
import re
import jieba
jieba.initialize()

# data = pd.read_csv("./off_line_file/article_data_v5_2017-03-01-20-56.txt", sep="\t", encoding="utf8")
input_file = "./off_line_file/article_data_v4_2017-02-17-16-59_part.csv"
data = pd.read_csv(input_file, sep="\t", encoding="gbk")

Building prefix dict from the default dictionary ...
Loading model from cache c:\users\wangwe~1\appdata\local\temp\jieba.cache
Loading model cost 0.524 seconds.
Prefix dict has been built succesfully.


In [ ]:
%%timeit
# %timeit 
data[u"pro_id1"] = data[u"pro_id"].fillna(value=-1).map(lambda id: str(id) if id != -1 else "")
vect_word.fit_transform(data["pro_id1"])

In [ ]:
%%timeit
data[u"pro_id2"] = data[u"pro_id"].where(data[u"pro_id"].notnull(), None)
transform(data["pro_id2"])

In [145]:
data = pd.read_csv("./off_line_file/train_data/article_data_v5_2017-03-01-20-56.txt", sep="\t", encoding="utf8")

In [144]:
%%timeit

data[u"pro_id"] = data[u"pro_id"].fillna(value="").astype(unicode)
data[u"level_1_id"] = data[u"level_1_id"].fillna(value="").astype(unicode)
data[u"level_2_id"] = data[u"level_2_id"].fillna(value="").astype(unicode)
data[u"level_3_id"] = data[u"level_3_id"].fillna(value="").astype(unicode)
data[u"level_4_id"] = data[u"level_4_id"].fillna(value="").astype(unicode)
data[u"brand_id"] = data[u"brand_id"].fillna(value="").astype(unicode)

cols = [u"pro_id", u"level_1_id", u"level_2_id", u"level_3_id", u"level_4_id", u"brand_id"]
for col in cols:
    vect = MyCountVectorizer(stop_words=("", "0", "-1"))
    vect.fit_transform(data[col])

10 loops, best of 3: 33.8 ms per loop


In [136]:
from sklearn.feature_extraction.text import CountVectorizer

In [146]:
%%timeit

data[u"pro_id"] = data[u"pro_id"].fillna(value=-1).map(lambda id: str(id) if id != -1 else "")
data[u"level_1_id"] = data[u"level_1_id"].map(lambda id: str(id) if id != -1 else "")
data[u"level_2_id"] = data[u"level_2_id"].map(lambda id: str(id) if id != -1 else "")
data[u"level_3_id"] = data[u"level_3_id"].map(lambda id: str(id) if id != -1 else "")
data[u"level_4_id"] = data[u"level_4_id"].map(lambda id: str(id) if id != -1 else "")
data[u"brand_id"] = data[u"brand_id"].fillna(value=-1).map(lambda id: str(id) if id != -1 else "")

cols = [u"pro_id", u"level_1_id", u"level_2_id", u"level_3_id", u"level_4_id", u"brand_id"]
for col in cols:
    vect = CountVectorizer(analyzer="word")
    vect.fit_transform(data[col])

10 loops, best of 3: 83.3 ms per loop


In [54]:
%%timeit
from sklearn.feature_extraction.text import CountVectorizer
vect_word = CountVectorizer(analyzer="word")

# data[u"pro_id"] = data[u"pro_id"].fillna(value=-1).map(lambda id: str(id) if id != -1 else "")
# data[u"level_1_id"] = data[u"level_1_id"].map(lambda id: str(id) if id != -1 else "")
# data[u"level_2_id"] = data[u"level_2_id"].map(lambda id: str(id) if id != -1 else "")
# data[u"level_3_id"] = data[u"level_3_id"].map(lambda id: str(id) if id != -1 else "")
# data[u"level_4_id"] = data[u"level_4_id"].map(lambda id: str(id) if id != -1 else "")
# data[u"brand_id"] = data[u"brand_id"].fillna(value=-1).map(lambda id: str(id) if id != -1 else "")
# data[u"level_3_id_is_null"] = np.where(data[u"level_3_id"] == "", 1, 0)


# vect_word.fit_transform(data[u"pro_id"])
# vect_word.fit_transform(data[u"level_1_id"])
# vect_word.fit_transform(data[u"level_2_id"])
# vect_word.fit_transform(data[u"level_3_id"])
# vect_word.fit_transform(data[u"level_4_id"])
# vect_word.fit_transform(data[u"brand_id"])
data[u"title"] = data[u"title"].map(lambda s: re.sub("\n", " ", s.upper()) if s else "")
sen = "\n".join(data[u"title"].values)
data[u"title_fc"] = ",".join(jieba.cut(sen)).split("\n")  # 对标题进行分词
# vect_word.fit_transform(data[u"title_fc"])

1 loop, best of 3: 395 ms per loop


In [132]:
import array
from collections import defaultdict, Mapping

import re
# import jieba
# jieba.initialize()
import numpy as np
import scipy.sparse as sp

from numpy import frombuffer as frombuffer_empty
from tools.stop_words import SIGNAL_STOP_WORDS, ENGLISH_STOP_WORDS

def _make_int_array():
    """Construct an array.array of a type suitable for scipy.sparse indices."""
    return array.array(str("i"))

class MyCountVectorizer(object):
    """
    将文档集转为一个 token 计数矩阵，矩阵采用 scipy.sparse.coo_matrix 稀疏矩阵。.
    文档集中每个文档都只包含一个词或者一个数字。

    Parameters
    ----------

    stop_words : string {'english'}, list, or None (default)
        If 'english', a built-in stop word list for English is used.

        If a list, that list is assumed to contain stop words, all of which
        will be removed from the resulting tokens.
        Only applies if ``analyzer == 'word'``.

        If None, no stop words will be used. max_df can be set to a value
        in the range [0.7, 1.0) to automatically detect and filter stop
        words based on intra corpus document frequency of terms.

    vocabulary : Mapping or iterable, optional
        Either a Mapping (e.g., a dict) where keys are terms and values are
        indices in the feature matrix, or an iterable over terms. If not
        given, a vocabulary is determined from the input documents. Indices
        in the mapping should not be repeated and should not have any gap
        between 0 and the largest index.

    binary : boolean, default=False
        If True, all non zero counts are set to 1. This is useful for discrete
        probabilistic models that model binary events rather than integer
        counts.

    dtype : type, optional
        Type of the matrix returned by fit_transform() or transform().

    Attributes
    ----------
    vocabulary_ : dict
        A mapping of terms to feature indices.

    stop_words_ : set
        Terms that were ignored because they either:

          - occurred in too many documents (`max_df`)
          - occurred in too few documents (`min_df`)
          - were cut off by feature selection (`max_features`).

        This is only available if no vocabulary was given.

    Notes
    -----
    The ``stop_words_`` attribute can get large and increase the model size
    when pickling. This attribute is provided only for introspection and can
    be safely removed using delattr or set to None before pickling.
    """

    def __init__(self, stop_words=None,
                 vocabulary=None, binary=False,
                 dtype=np.int8):
        self.stop_words = stop_words
        self.vocabulary = vocabulary
        self.binary = binary
        self.dtype = dtype

    def _validate_vocabulary(self):
        vocabulary = self.vocabulary
        # vocabulary 不为空
        if vocabulary is not None:
            if isinstance(vocabulary, set):
                vocabulary = sorted(vocabulary)
            if not isinstance(vocabulary, Mapping):
                vocab = {}
                for i, t in enumerate(vocabulary):
                    if vocab.setdefault(t, i) != i:
                        msg = "Duplicate term in vocabulary: %r" % t
                        raise ValueError(msg)
                vocabulary = vocab
            else:
                indices = set(vocabulary.itervalues())
                if len(indices) != len(vocabulary):
                    raise ValueError("Vocabulary contains repeated indices.")
                for i in xrange(len(vocabulary)):
                    if i not in indices:
                        msg = ("Vocabulary of size %d doesn't contain index "
                               "%d." % (len(vocabulary), i))
                        raise ValueError(msg)
            if not vocabulary:
                raise ValueError("empty vocabulary passed to fit")
            self.fixed_vocabulary_ = True
            self.vocabulary_ = dict(vocabulary)
        else:
            self.fixed_vocabulary_ = False

    def _sort_features(self, X, vocabulary):
        """Sort features by name

        Returns a reordered matrix and modifies the vocabulary in place
        """
        sorted_features = sorted(vocabulary.iteritems())
        map_index = np.empty(len(sorted_features), dtype=np.int32)
        for new_val, (term, old_val) in enumerate(sorted_features):
            map_index[new_val] = old_val
            vocabulary[term] = new_val
        return X[:, map_index]

    def _count_vocab(self, raw_documents, fixed_vocab):
        """Create sparse feature matrix, and vocabulary where fixed_vocab=False
        """
        if fixed_vocab:
            vocabulary = self.vocabulary_
        else:
            # Add a new value when a new vocabulary item is seen
            vocabulary = defaultdict()
            vocabulary.default_factory = vocabulary.__len__

        j_indices = []
        indptr = _make_int_array()
        values = _make_int_array()
        indptr.append(0)

        for doc in raw_documents:
            if self.stop_words:
                if doc in self.stop_words:
                    indptr.append(len(j_indices))
                    continue
            feature_counter = {}
            try:
                feature_idx = vocabulary[doc]
                if feature_idx not in feature_counter:
                    feature_counter[feature_idx] = 1
                else:
                    feature_counter[feature_idx] += 1
            except KeyError:
                # 使用自定义词库表时，如果文档中包含的词汇[特征]不在自定义词库表中，
                # 则不再将该词汇[特征]加入到词库表
                # Ignore out-of-vocabulary items for fixed_vocab=True
                continue

            j_indices.extend(feature_counter.keys())
            values.extend(feature_counter.values())
            indptr.append(len(j_indices))

        # 自动生成[学习]词库表的情况
        if not fixed_vocab:
            # disable defaultdict behaviour
            vocabulary = dict(vocabulary)
            if not vocabulary:
                raise ValueError("empty vocabulary; perhaps the documents only"
                                 " contain stop words")

        j_indices = np.asarray(j_indices, dtype=np.intc)
        indptr = np.frombuffer(indptr, dtype=np.intc)
        values = frombuffer_empty(values, dtype=np.intc)

        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocabulary)),
                          dtype=self.dtype)
        #
        X.sort_indices()
        return vocabulary, X

    def fit_transform(self, raw_documents):
        """Learn the vocabulary dictionary and return term-document matrix.

        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.

        Returns
        -------
        X : array, [n_samples, n_features]
            Document-term matrix.
        """
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")

        # 校验vocabulary，如果之前提供了自定义vocabulary，则将self.fixed_vocabulary_设为True，否则False
        self._validate_vocabulary()

        vocabulary, X = self._count_vocab(raw_documents,
                                          self.fixed_vocabulary_)

        if self.binary:
            X.data.fill(1)

        # 自动生成[学习]词库表的情况
        if not self.fixed_vocabulary_:
            X = self._sort_features(X, vocabulary)
            self.vocabulary_ = vocabulary
        return X

    def inverse_transform(self, X):
        """Return terms per document with nonzero entries in X.

        Parameters
        ----------
        X : {array, sparse matrix}, shape = [n_samples, n_features]

        Returns
        -------
        X_inv : list of arrays, len = n_samples
            List of arrays of terms.
        """

        if sp.issparse(X):
            # We need CSR format for fast row manipulations.
            X = X.tocsr()
        else:
            # We need to convert X to a matrix, so that the indexing
            # returns 2D objects
            X = np.asmatrix(X)
        n_samples = X.shape[0]

        terms = np.array(list(self.vocabulary_.keys()))
        indices = np.array(list(self.vocabulary_.values()))
        inverse_vocabulary = terms[np.argsort(indices)]

        return [inverse_vocabulary[X[i, :].nonzero()[1]].ravel()
                for i in range(n_samples)]

    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""

        # return self.vocabulary_.keys()
        return [k for k, v in sorted(self.vocabulary_.iteritems(), key=lambda item: item[1])]

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

In [78]:
%%timeit

vect =  CountVectorizer(analyzer="char")
vect.fit_transform(data.crowd)

100 loops, best of 3: 11.4 ms per loop


In [79]:
%%timeit

vect =  CountVectorizer(analyzer="word", tokenizer=lambda x: x)
vect.fit_transform(data.crowd)

100 loops, best of 3: 9.17 ms per loop


In [87]:
%%timeit

vect =  CountVectorizer(analyzer="word", preprocessor=lambda x: x)
m = vect.fit_transform(data.brand_id)

100 loops, best of 3: 11.8 ms per loop


In [114]:
# %%timeit
vect =  CountVectorizer(analyzer="word")
docs = ["123", "456", "145"]
m = vect.fit_transform(data.brand_id)
# print m.toarray()
print vect.get_feature_names()

[u'10005', u'1001', u'1003', u'10063', u'10123', u'10211', u'1023', u'10255', u'1029', u'1031', u'1033', u'1041', u'1045', u'1047', u'1049', u'10533', u'10625', u'1067', u'107', u'1071', u'1075', u'10773', u'1079', u'1083', u'1087', u'1089', u'1099', u'1101', u'1103', u'11055', u'11063', u'1107', u'1109', u'1119', u'11241', u'11327', u'1133', u'11367', u'1139', u'11397', u'1141', u'11467', u'11553', u'11589', u'1159', u'1163', u'1165', u'11683', u'1173', u'11741', u'1175', u'1177', u'1179', u'11859', u'1187', u'1189', u'11949', u'1197', u'11979', u'1199', u'12019', u'1203', u'1205', u'1213', u'12181', u'12195', u'12257', u'12301', u'1233', u'12365', u'1239', u'12461', u'12521', u'1253', u'1273', u'12731', u'12867', u'12869', u'12883', u'1291', u'13', u'13021', u'13145', u'1329', u'13305', u'13313', u'1353', u'13647', u'1367', u'13709', u'1383', u'13833', u'1391', u'13935', u'1395', u'1397', u'1409', u'14123', u'1417', u'1461', u'14653', u'14681', u'14701', u'1471', u'14979', u'1503', u

In [130]:
class MyCountVectorizer(object):
    """
    将文档集转为一个 token 计数矩阵，矩阵采用 scipy.sparse.coo_matrix 稀疏矩阵。.

    If you do not provide an a-priori dictionary and you do not use an analyzer
    that does some kind of feature selection then the number of features will
    be equal to the vocabulary size found by analyzing the data.


    Parameters
    ----------
    analyzer : string, {'word', 'char'} or callable
        Whether the feature should be made of word or character n-grams.
        Option 'char_wb' creates character n-grams only from text inside
        word boundaries.

        If a callable is passed it is used to extract the sequence of features
        out of the raw, unprocessed input.

    preprocessor : callable or None (default)
        Override the preprocessing (string transformation) stage while
        preserving the tokenizing and n-grams generation steps.

    tokenizer : callable or None (default)
        Override the string tokenization step while preserving the
        preprocessing and n-grams generation steps.
        Only applies if ``analyzer == 'word'``.

    stop_words : string {'english'}, list, or None (default)
        If 'english', a built-in stop word list for English is used.

        If a list, that list is assumed to contain stop words, all of which
        will be removed from the resulting tokens.
        Only applies if ``analyzer == 'word'``.

        If None, no stop words will be used. max_df can be set to a value
        in the range [0.7, 1.0) to automatically detect and filter stop
        words based on intra corpus document frequency of terms.

    lowercase : boolean, True by default
        Convert all characters to lowercase before tokenizing.

    token_pattern : string
        Regular expression denoting what constitutes a "token", only used
        if ``analyzer == 'word'``. The default regexp select tokens of 2
        or more alphanumeric characters (punctuation is completely ignored
        and always treated as a token separator).

    vocabulary : Mapping or iterable, optional
        Either a Mapping (e.g., a dict) where keys are terms and values are
        indices in the feature matrix, or an iterable over terms. If not
        given, a vocabulary is determined from the input documents. Indices
        in the mapping should not be repeated and should not have any gap
        between 0 and the largest index.

    binary : boolean, default=False
        If True, all non zero counts are set to 1. This is useful for discrete
        probabilistic models that model binary events rather than integer
        counts.

    dtype : type, optional
        Type of the matrix returned by fit_transform() or transform().

    doc_type : str, {'number', 'str'}, default='str'

    is_record_analyzer_res : boolean, default=False

    Attributes
    ----------
    vocabulary_ : dict
        A mapping of terms to feature indices.

    stop_words_ : set
        Terms that were ignored because they either:

          - occurred in too many documents (`max_df`)
          - occurred in too few documents (`min_df`)
          - were cut off by feature selection (`max_features`).

        This is only available if no vocabulary was given.

    Notes
    -----
    The ``stop_words_`` attribute can get large and increase the model size
    when pickling. This attribute is provided only for introspection and can
    be safely removed using delattr or set to None before pickling.
    """

    def __init__(self, lowercase=True, stop_words=None,
                 vocabulary=None, binary=False,
                 dtype=np.int8):
        self.lowercase = lowercase
        self.stop_words = stop_words
        self.vocabulary = vocabulary
        self.binary = binary
        self.dtype = dtype

    def _validate_vocabulary(self):
        vocabulary = self.vocabulary
        # vocabulary 不为空
        if vocabulary is not None:
            if isinstance(vocabulary, set):
                vocabulary = sorted(vocabulary)
            if not isinstance(vocabulary, Mapping):
                vocab = {}
                for i, t in enumerate(vocabulary):
                    if vocab.setdefault(t, i) != i:
                        msg = "Duplicate term in vocabulary: %r" % t
                        raise ValueError(msg)
                vocabulary = vocab
            else:
                indices = set(vocabulary.itervalues())
                if len(indices) != len(vocabulary):
                    raise ValueError("Vocabulary contains repeated indices.")
                for i in xrange(len(vocabulary)):
                    if i not in indices:
                        msg = ("Vocabulary of size %d doesn't contain index "
                               "%d." % (len(vocabulary), i))
                        raise ValueError(msg)
            if not vocabulary:
                raise ValueError("empty vocabulary passed to fit")
            self.fixed_vocabulary_ = True
            self.vocabulary_ = dict(vocabulary)
        else:
            self.fixed_vocabulary_ = False

    def _sort_features(self, X, vocabulary):
        """Sort features by name

        Returns a reordered matrix and modifies the vocabulary in place
        """
        sorted_features = sorted(vocabulary.iteritems())
        map_index = np.empty(len(sorted_features), dtype=np.int32)
        for new_val, (term, old_val) in enumerate(sorted_features):
            map_index[new_val] = old_val
            vocabulary[term] = new_val
        return X[:, map_index]

    def _count_vocab(self, raw_documents, fixed_vocab):
        """Create sparse feature matrix, and vocabulary where fixed_vocab=False
        """
        if fixed_vocab:
            vocabulary = self.vocabulary_
        else:
            # Add a new value when a new vocabulary item is seen
            vocabulary = defaultdict()
            vocabulary.default_factory = vocabulary.__len__

        j_indices = []
        indptr = _make_int_array()
        values = _make_int_array()
        indptr.append(0)

        for doc in raw_documents:
            if self.stop_words:
                if doc in self.stop_words:
                    indptr.append(len(j_indices))
                    continue
            feature_counter = {}
            try:
                feature_idx = vocabulary[doc]
                if feature_idx not in feature_counter:
                    feature_counter[feature_idx] = 1
                else:
                    feature_counter[feature_idx] += 1
            except KeyError:
                # 使用自定义词库表时，如果文档中包含的词汇[特征]不在自定义词库表中，
                # 则不再将该词汇[特征]加入到词库表
                # Ignore out-of-vocabulary items for fixed_vocab=True
                continue

            j_indices.extend(feature_counter.keys())
            values.extend(feature_counter.values())
            indptr.append(len(j_indices))

        # 自动生成[学习]词库表的情况
        if not fixed_vocab:
            # disable defaultdict behaviour
            vocabulary = dict(vocabulary)
            if not vocabulary:
                raise ValueError("empty vocabulary; perhaps the documents only"
                                 " contain stop words")

        j_indices = np.asarray(j_indices, dtype=np.intc)
        indptr = np.frombuffer(indptr, dtype=np.intc)
        values = frombuffer_empty(values, dtype=np.intc)

        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocabulary)),
                          dtype=self.dtype)
        #
        X.sort_indices()
        return vocabulary, X

    def fit_transform(self, raw_documents):
        """Learn the vocabulary dictionary and return term-document matrix.

        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.

        Returns
        -------
        X : array, [n_samples, n_features]
            Document-term matrix.
        """
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")

        # 校验vocabulary，如果之前提供了自定义vocabulary，则将self.fixed_vocabulary_设为True，否则False
        self._validate_vocabulary()

        vocabulary, X = self._count_vocab(raw_documents,
                                          self.fixed_vocabulary_)

        if self.binary:
            X.data.fill(1)

        # 自动生成[学习]词库表的情况
        if not self.fixed_vocabulary_:
            X = self._sort_features(X, vocabulary)
            self.vocabulary_ = vocabulary
        return X

    def inverse_transform(self, X):
        """Return terms per document with nonzero entries in X.

        Parameters
        ----------
        X : {array, sparse matrix}, shape = [n_samples, n_features]

        Returns
        -------
        X_inv : list of arrays, len = n_samples
            List of arrays of terms.
        """

        if sp.issparse(X):
            # We need CSR format for fast row manipulations.
            X = X.tocsr()
        else:
            # We need to convert X to a matrix, so that the indexing
            # returns 2D objects
            X = np.asmatrix(X)
        n_samples = X.shape[0]

        terms = np.array(list(self.vocabulary_.keys()))
        indices = np.array(list(self.vocabulary_.values()))
        inverse_vocabulary = terms[np.argsort(indices)]

        return [inverse_vocabulary[X[i, :].nonzero()[1]].ravel()
                for i in range(n_samples)]

    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""

        # return self.vocabulary_.keys()
        return [k for k, v in sorted(self.vocabulary_.iteritems(), key=lambda item: item[1])]

In [98]:
d = data.brand_id.astype(int)

In [128]:
# %%timeit
vect = MyCountVectorizer(stop_words=['0'])
docs = ["123", "456", "145", "0"]
m = vect.fit_transform(docs)
print m.toarray()
print vect.get_feature_names()


[[0 1 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [1 0 0 0]]
['0', '123', '145', '456']


In [104]:
docs = ["123", "456", "145"]
vect =  CountVectorizer(analyzer="word", preprocessor=lambda x: x)
m = vect.fit_transform(docs)
print m.toarray()
print vect.get_feature_names()

[[1 0 0]
 [0 0 1]
 [0 1 0]]
[u'123', u'145', u'456']


In [14]:
docs = ["达斯登 速干T恤男",
"达斯登 速干T恤男 女短袖",
"速干T恤男 女短袖",
"达斯登 速干T恤男 女短袖"
]
m1 = Vectorizer1(doc_type="str", analyzer="word", stop_words=[" "],
                    preprocessor=None, tokenizer=jieba.cut, is_record_analyzer_res=True)
print m1.fit_transform(docs).toarray()
print ", ".join(m1.get_feature_names())

[[1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1]
 [0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
达, 斯登, 速干, t, 恤, 男, 女, 短袖


In [55]:
m2 = Vectorizer2(analyzer="word", stop_words=[" "], preprocessor=None, tokenizer=jieba.cut)
print m2.fit_transform(docs).toarray()
print ", ".join(m2.get_feature_names())
m = m2.fit_transform(docs)
m.toarray()

[[0 1]
 [1 0]
 [0 0]]
女, 男


array([[0, 1],
       [1, 0],
       [0, 0]], dtype=int64)

In [93]:
d = m2.inverse_transform([0,1])
d = map(lambda x: ", ".join(x), d)
print "\n".join(d)

男


In [62]:
pd.Series([",".join(t) for t in vect_word.documents_analyzer_res]).head()

0                                蓝迪,智慧,乐园,儿童,磁性,彩色,绘画板
1                        reen,润膏,蓝色,燕窝,洗发水,250ml,*,3,件
2         tcl,kfrd,-,35gw,/,ew13bpa,1.5,匹,变频,冷暖,壁挂式,空调
3    海信,（,hisense,）,bcd,-,321wt,/,q,321,升,风冷,无霜,节能,...
4                            triumph,黛安芬,e002208,女士,文胸
dtype: object

In [36]:
docs = ["男",
"女",
""
]
m2 = Vectorizer2(analyzer="word", stop_words=[" "], preprocessor=None, tokenizer=lambda x: x)
print m2.fit_transform(docs).toarray()
print ", ".join(m2.get_feature_names())

[[0 1]
 [1 0]
 [0 0]]
女, 男


In [94]:
import array
from collections import defaultdict, Mapping

import re
# import jieba
# jieba.initialize()
import numpy as np
import scipy.sparse as sp

from numpy import frombuffer as frombuffer_empty

def _make_int_array():
    """Construct an array.array of a type suitable for scipy.sparse indices."""
    return array.array(str("i"))


class MyCountVectorizer(object):
    """
    将文档集转为一个 token 计数矩阵，矩阵采用 scipy.sparse.coo_matrix 稀疏矩阵。.

    If you do not provide an a-priori dictionary and you do not use an analyzer
    that does some kind of feature selection then the number of features will
    be equal to the vocabulary size found by analyzing the data.


    Parameters
    ----------
    analyzer : string, {'word', 'char'} or callable
        Whether the feature should be made of word or character n-grams.
        Option 'char_wb' creates character n-grams only from text inside
        word boundaries.

        If a callable is passed it is used to extract the sequence of features
        out of the raw, unprocessed input.

    preprocessor : callable or None (default)
        Override the preprocessing (string transformation) stage while
        preserving the tokenizing and n-grams generation steps.

    tokenizer : callable or None (default)
        Override the string tokenization step while preserving the
        preprocessing and n-grams generation steps.
        Only applies if ``analyzer == 'word'``.

    stop_words : string {'english'}, list, or None (default)
        If 'english', a built-in stop word list for English is used.

        If a list, that list is assumed to contain stop words, all of which
        will be removed from the resulting tokens.
        Only applies if ``analyzer == 'word'``.

        If None, no stop words will be used. max_df can be set to a value
        in the range [0.7, 1.0) to automatically detect and filter stop
        words based on intra corpus document frequency of terms.

    lowercase : boolean, True by default
        Convert all characters to lowercase before tokenizing.

    token_pattern : string
        Regular expression denoting what constitutes a "token", only used
        if ``analyzer == 'word'``. The default regexp select tokens of 2
        or more alphanumeric characters (punctuation is completely ignored
        and always treated as a token separator).

    vocabulary : Mapping or iterable, optional
        Either a Mapping (e.g., a dict) where keys are terms and values are
        indices in the feature matrix, or an iterable over terms. If not
        given, a vocabulary is determined from the input documents. Indices
        in the mapping should not be repeated and should not have any gap
        between 0 and the largest index.

    binary : boolean, default=False
        If True, all non zero counts are set to 1. This is useful for discrete
        probabilistic models that model binary events rather than integer
        counts.

    dtype : type, optional
        Type of the matrix returned by fit_transform() or transform().

    doc_type : str, {'number', 'str'}, default='str'

    is_record_analyzer_res : boolean, default=False

    Attributes
    ----------
    vocabulary_ : dict
        A mapping of terms to feature indices.

    stop_words_ : set
        Terms that were ignored because they either:

          - occurred in too many documents (`max_df`)
          - occurred in too few documents (`min_df`)
          - were cut off by feature selection (`max_features`).

        This is only available if no vocabulary was given.

    Notes
    -----
    The ``stop_words_`` attribute can get large and increase the model size
    when pickling. This attribute is provided only for introspection and can
    be safely removed using delattr or set to None before pickling.
    """

    def __init__(self, lowercase=True, stop_words=None,
                 vocabulary=None, binary=False,
                 dtype=np.int8):
        self.lowercase = lowercase
        self.stop_words = stop_words
        self.vocabulary = vocabulary
        self.binary = binary
        self.dtype = dtype

    def _validate_vocabulary(self):
        vocabulary = self.vocabulary
        # vocabulary 不为空
        if vocabulary is not None:
            if isinstance(vocabulary, set):
                vocabulary = sorted(vocabulary)
            if not isinstance(vocabulary, Mapping):
                vocab = {}
                for i, t in enumerate(vocabulary):
                    if vocab.setdefault(t, i) != i:
                        msg = "Duplicate term in vocabulary: %r" % t
                        raise ValueError(msg)
                vocabulary = vocab
            else:
                indices = set(vocabulary.itervalues())
                if len(indices) != len(vocabulary):
                    raise ValueError("Vocabulary contains repeated indices.")
                for i in xrange(len(vocabulary)):
                    if i not in indices:
                        msg = ("Vocabulary of size %d doesn't contain index "
                               "%d." % (len(vocabulary), i))
                        raise ValueError(msg)
            if not vocabulary:
                raise ValueError("empty vocabulary passed to fit")
            self.fixed_vocabulary_ = True
            self.vocabulary_ = dict(vocabulary)
        else:
            self.fixed_vocabulary_ = False

    def _sort_features(self, X, vocabulary):
        """Sort features by name

        Returns a reordered matrix and modifies the vocabulary in place
        """
        sorted_features = sorted(vocabulary.iteritems())
        map_index = np.empty(len(sorted_features), dtype=np.int32)
        for new_val, (term, old_val) in enumerate(sorted_features):
            map_index[new_val] = old_val
            vocabulary[term] = new_val
        return X[:, map_index]

    def _count_vocab(self, raw_documents, fixed_vocab):
        """Create sparse feature matrix, and vocabulary where fixed_vocab=False
        """
        if fixed_vocab:
            vocabulary = self.vocabulary_
        else:
            # Add a new value when a new vocabulary item is seen
            vocabulary = defaultdict()
            vocabulary.default_factory = vocabulary.__len__

        j_indices = []
        indptr = _make_int_array()
        values = _make_int_array()
        indptr.append(0)

        for doc in raw_documents:
            if doc in self.stop_words:
                indptr.append(len(j_indices))
                continue
            feature_counter = {}
            try:
                feature_idx = vocabulary[doc]
                if feature_idx not in feature_counter:
                    feature_counter[feature_idx] = 1
                else:
                    feature_counter[feature_idx] += 1
            except KeyError:
                # 使用自定义词库表时，如果文档中包含的词汇[特征]不在自定义词库表中，
                # 则不再将该词汇[特征]加入到词库表
                # Ignore out-of-vocabulary items for fixed_vocab=True
                continue

            j_indices.extend(feature_counter.keys())
            values.extend(feature_counter.values())
            indptr.append(len(j_indices))

        # 自动生成[学习]词库表的情况
        if not fixed_vocab:
            # disable defaultdict behaviour
            vocabulary = dict(vocabulary)
            if not vocabulary:
                raise ValueError("empty vocabulary; perhaps the documents only"
                                 " contain stop words")

        j_indices = np.asarray(j_indices, dtype=np.intc)
        indptr = np.frombuffer(indptr, dtype=np.intc)
        values = frombuffer_empty(values, dtype=np.intc)

        X = sp.csr_matrix((values, j_indices, indptr),
                          shape=(len(indptr) - 1, len(vocabulary)),
                          dtype=self.dtype)
        #
        X.sort_indices()
        return vocabulary, X

    def fit_transform(self, raw_documents):
        """Learn the vocabulary dictionary and return term-document matrix.

        Parameters
        ----------
        raw_documents : iterable
            An iterable which yields either str, unicode or file objects.

        Returns
        -------
        X : array, [n_samples, n_features]
            Document-term matrix.
        """
        if isinstance(raw_documents, str):
            raise ValueError(
                "Iterable over raw text documents expected, "
                "string object received.")

        # 校验vocabulary，如果之前提供了自定义vocabulary，则将self.fixed_vocabulary_设为True，否则False
        self._validate_vocabulary()

        vocabulary, X = self._count_vocab(raw_documents,
                                          self.fixed_vocabulary_)

        if self.binary:
            X.data.fill(1)

        # 自动生成[学习]词库表的情况
        if not self.fixed_vocabulary_:
            X = self._sort_features(X, vocabulary)
            self.vocabulary_ = vocabulary
        return X

    def inverse_transform(self, X):
        """Return terms per document with nonzero entries in X.

        Parameters
        ----------
        X : {array, sparse matrix}, shape = [n_samples, n_features]

        Returns
        -------
        X_inv : list of arrays, len = n_samples
            List of arrays of terms.
        """

        if sp.issparse(X):
            # We need CSR format for fast row manipulations.
            X = X.tocsr()
        else:
            # We need to convert X to a matrix, so that the indexing
            # returns 2D objects
            X = np.asmatrix(X)
        n_samples = X.shape[0]

        terms = np.array(list(self.vocabulary_.keys()))
        indices = np.array(list(self.vocabulary_.values()))
        inverse_vocabulary = terms[np.argsort(indices)]

        return [inverse_vocabulary[X[i, :].nonzero()[1]].ravel()
                for i in range(n_samples)]

    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""

        # return self.vocabulary_.keys()
        return [k for k, v in sorted(self.vocabulary_.iteritems(), key=lambda item: item[1])]

In [ ]:
data = pd.read_csv("./off_line_file/article_data_v5_2017-03-01-20-56.txt", sep="\t", encoding="utf8")

# model_main = ModelMain(data, features_with_weight_1,
#                        features_with_weight_2, src_article_num=None,
#                        rec_article_num=3, title_fc_extra_weight=None,
#                        rec_format="list", ndigits=2)
# r1 = model_main.run_similary()
pp = PreProcess()
data = pp(data)
model = SimilaryModel(data, features_with_weight_1,
                      features_with_weight_2, src_article_num=None,
                      rec_article_num=3, title_fc_extra_weight=None,
                      ndigits=2)
res_csr_matrix = model.calculate_similary()
res = model.map_articles(res_csr_matrix, res_format="list")

In [ ]:
# coding=utf-8
import datetime
import pandas as pd
import time

from model_title_fenci_weight_up.model import ModelMain, features_with_weight_1, features_with_weight_2, SimilaryModel
from model_title_fenci_weight_up.preprocessing import PreProcess

def construct_frame(res_list, res_cols):
    return pd.DataFrame(data=res_list, columns=res_cols)


def add_article_attrs(dataframe, associate_attrs, flag="src"):
    left_on_key = "src_article" if flag=="src" else "rec_article"
    src_col_list = dataframe.columns.tolist()
    src_col_list.extend(associate_attrs)
    dataframe = dataframe.merge(data, left_on=left_on_key, right_on="article_id")
    cols_map = dict(zip(associate_attrs, map(lambda s: flag + "_" + s, associate_attrs)))
    dataframe.rename_axis(cols_map, axis=1, inplace=True)
    return dataframe

In [ ]:
res_cols = [u"src_article", u"rec_article", u"score_sum",
        u"score_pro", u"score_level1", u"score_level2",
        u"score_level3", u"score_level4", u"score_brand",
        u"score_title_fc", u"score_sex", u"score_crowd"]
associate_cols = ["pro_id", "level_1", "level_2", "level_3", "level_4", "brand", "title", "title_fc", "pubdate"]

recommend = construct_frame(res, res_cols)
recommend = add_article_attrs(recommend, associate_cols, flag="src")
print recommend.columns
recommend = add_article_attrs(recommend, associate_cols, flag="rec")
# recommend.head()

In [ ]:
sorted_cols = [u'src_article', u'rec_article', u'src_title', u'rec_title', u'score_sum', u'score_pro', u'src_pro_id',  u'rec_pro_id',
 u'score_level4', u'src_level_4', u'rec_level_4', u'score_level3', u'src_level_3', u'rec_level_3',
 u'score_level2', u'src_level_2', u'rec_level_2', u'score_level1', u'src_level_1', u'rec_level_1',
 u'score_brand', u'src_brand', u'rec_brand', u'score_title_fc', u'src_title_fc', u'rec_title_fc', u'score_sex', u'score_crowd']
recommend = recommend[sorted_cols]
recommend = recommend.sort_values(by=["src_article","score_sum"], ascending=False)
recommend.head()